In [1]:
from PIL import Image, ImageDraw
import random
import numpy as np

In [2]:
# read image
im = Image.open('cap1.png')
#im = Image.open('test.jpg')
Vsize, Hsize = im.size
VnumArray = np.arange(Vsize)
HnumArray = np.arange(Hsize)
boxLen = 0
boxWid = 0
print Vsize, Hsize
im.show()

1920 962


In [3]:
# convert image data to numpy format
imPIX = im.load()
imDATA = np.zeros((3, Hsize, Vsize))

for x in range(Hsize):
    for y in range(Vsize):
        imDATA[0][x,y] = imPIX[y,x][0]
        imDATA[1][x,y] = imPIX[y,x][1]
        imDATA[2][x,y] = imPIX[y,x][2]

In [4]:
# quantization
def quantization(data, stat=np.array([0,85,170,255])):
    delta = data - stat
    delta = abs(delta)
    mindelta = delta.min()
    return stat[delta == mindelta]

#for x in range(Hsize):
#    for y in range(Vsize):
#        imDATA[0][x,y] = quantization(imDATA[0][x,y])
#        imDATA[1][x,y] = quantization(imDATA[1][x,y])
#        imDATA[2][x,y] = quantization(imDATA[2][x,y])
#

In [5]:
### test
#data = np.array([78])
#stat=np.array([0,85,170,255])
#delta = data - stat
#delta = abs(delta)
#mindelta = delta.min()
#delta == mindelta
##stat[delta == mindelta]

In [50]:
# correct

In [6]:


imDATA[0].mean(axis=0)

array([ 9.94074844,  9.94074844,  9.94074844, ...,  9.94074844,
        9.94074844,  9.94074844])

In [7]:
a = imDATA[1].mean(axis=1)


In [8]:
# cal, LtUpPt,LtDnPt,LtDnPt,RtDnPt,X0,Y0,X1,Y1
#boxLen
#boxWid

# white row/col is belivable
whiteRow = HnumArray[imDATA[0].mean(axis=1) > 225]
whiteCol = VnumArray[imDATA[0].mean(axis=0) > 225]
# black is uncertainty because of the captured boarder
blackRow = HnumArray[imDATA[0].mean(axis=1) < 25]
blackCol = VnumArray[imDATA[0].mean(axis=0) < 25]

if whiteRow.shape == whiteCol.shape:
    boxLen = whiteRow.shape[0]/4
    print boxLen
else:
    raise Exception ("ERROR: whiteRow is not equal to whiteCol")

print whiteRow, whiteCol
Y0 = whiteCol[2*boxLen-1] + boxLen + 1
Y1 = whiteCol[2*boxLen]   - boxLen
X0 = whiteRow[2*boxLen-1] + boxLen + 1
X1 = whiteRow[2*boxLen]   - boxLen
print X0, X1, Y0,Y1
LtUpPt = (X0, Y0)
LtDnPt = (X1, Y0)
RtUpPt = (X0, Y1)
RtDnPt = (X1, Y1)
print LtUpPt, RtDnPt
if (X1-X0)%boxLen !=0 or (Y1-Y0)%boxLen !=0 :
    raise Exception("(X1-X0) mod boxLen = %d, (Y1-Y0) mod boxLen=%d" % ((X1-X0)%boxLen, (Y1-Y0)%boxLen))


7
[  9  10  11  12  13  14  15  24  25  26  27  28  29  30 931 932 933 934
 935 936 937 946 947 948 949 950 951 952] [   8    9   10   11   12   13   14   23   24   25   26   27   28   29 1890
 1891 1892 1893 1894 1895 1896 1905 1906 1907 1908 1909 1910 1911]
38 924 37 1883
(38, 37) (924, 1883)


Exception: (X1-X0) mod boxLen = 4, (Y1-Y0) mod boxLen=5

In [38]:
print X1-X0, Y1-Y0

886 1846


In [9]:
tmp = imDATA.sum(axis=0)
#tmp.mean(axis=1) > 225*3
timingRow = HnumArray[whiteRow[boxLen-1]+1:whiteRow[boxLen]-1]
timingCol = HnumArray[whiteCol[boxLen-1]+1:whiteCol[boxLen]-1]
#timingRowPos = X0:X1, whiteRow[boxLen-1]+1:whiteRow[boxLen]
tmp1 = tmp[X0:X1,whiteRow[boxLen-1]+1:whiteRow[boxLen]-1]
tmp2 = tmp[Y0:Y1,whiteCol[boxLen-1]+1:whiteCol[boxLen]-1]

In [10]:
tmp1.shape

(886L, 7L)

In [11]:
tmp2.shape

(925L, 7L)

In [12]:
tmp1.sum(axis=1)>4500

array([False,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,

In [34]:
np.convolve(tmp2.sum(axis=1),[1,1,1,1,1,1,1])

array([  1785.,   5362.,  10717.,  16072.,  21427.,  26782.,  32137.,
        35707.,  37296.,  32157.,  26829.,  21501.,  16173.,  10845.,
         5517.,    378.,   2862.,   8190.,  13518.,  18846.,  24174.,
        29502.,  34830.,  37296.,  32157.,  26829.,  21501.,  16173.,
        10845.,   5517.,    378.,   2862.,   8190.,  13518.,  18846.,
        24174.,  29502.,  34830.,  37296.,  32157.,  26829.,  21501.,
        16173.,  10845.,   5517.,    378.,   2862.,   8190.,  13518.,
        18846.,  24174.,  29502.,  34830.,  37296.,  32157.,  26829.,
        21501.,  16173.,  10845.,   5517.,    378.,   2862.,   8190.,
        13518.,  18846.,  24174.,  29502.,  34830.,  37296.,  32157.,
        26829.,  21501.,  16173.,  10845.,   5517.,    378.,   2862.,
         8190.,  13518.,  18846.,  24174.,  29502.,  34830.,  37296.,
        32157.,  26829.,  21501.,  16173.,  10845.,   5517.,    378.,
         2862.,   8190.,  13518.,  18846.,  24174.,  29502.,  34830.,
        37296.,  321

In [ ]:
# check and correct each box
whiteRow

In [ ]:
print (X1-X0)%boxLen, (Y1-Y0)%boxLen

In [ ]:
whiteRow.shape[0]

In [45]:
draw=ImageDraw.Draw(im)
draw.line((0,0)+im.size, fill=128)
draw.line((Y0,0,Y0,im.size[1]), fill=128)
draw.line((Y1,0,Y1,im.size[1]), fill=128)
draw.line((0,X0,im.size[0],X0), fill=128)
draw.line((0,X1,im.size[0],X1), fill=128)

In [46]:
im.show()

In [42]:
# get pixel of each box
def getBoxPIX(imPayload, boxNum=(0,0), boxLen=4):
    boxData = np.zeros([3,boxLen, boxLen])
    i0 = boxNum[0]*boxLen
    i1 = (boxNum[0]+1)*boxLen
    j0 = boxNum[1]*boxLen
    j1 = (boxNum[1]+1)*boxLen
    boxData[0] = imPayload[0][i0:i1,j0:j1]
    boxData[1] = imPayload[1][i0:i1,j0:j1]
    boxData[2] = imPayload[2][i0:i1,j0:j1]
    print boxData
    return boxData[0].mean(), boxData[1].mean(), boxData[2].mean()


In [45]:
imPayload = np.zeros([3, X1-X0, Y1-Y0])
imPayload[0] = imDATA[0][X0:X1, Y0:Y1]
imPayload[1] = imDATA[1][X0:X1, Y0:Y1]
imPayload[2] = imDATA[2][X0:X1, Y0:Y1]
print imPayload.shape
getBoxPIX(imPayload, boxNum=(45,5), boxLen=boxLen)

(3L, 826L, 1722L)
[[[  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.    0.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.    0.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.  170.  170.   85.  170.   85.   85.]]

 [[  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [  85.   85.   85.   85.   85.   85.   85.]]

 [[ 170.  255.  255.  255.  255.  255.  170.]
  [ 255.  255.  255.  255.  255.  255.  170.]
  [ 170.  255.  255.  255.  255.  255.  170.]
  [ 255.  255.  255.  255.  255.  255.  170.]
  [ 255.  255.  255.  255.  255.  255.  255.]
  [ 170.  255.  255.  255.  255.  255.  170.]
  [ 170.  170.  170.  170.  170.  170.   85.]]]


(86.734693877551024, 85.0, 227.24489795918367)

In [ ]:
boxLen

In [ ]:
imPIX[19,19]

In [ ]:
imPIX[21,21]

In [ ]:
imPIX[22,22]

In [ ]:
imPIX[23,23]

In [ ]:
imPIX[24,24]

In [ ]:
imPIX[25,25]

In [ ]:
def isBlackLine():
    pass
def isWhiteLine():
    pass
def isTimingLine():
    pass
def isRtUpCorn():
    pass
def isRtDnCorn():
    pass
def isLtUpCorn():
    pass
def isLtDnCorn():
    pass
def getBoxLen():
    pass
def getPayloadRtUpPoint():
    pass
def getPayloadRtDnPoint():
    pass
def getPayloadLtUpPoint():
    pass
def getPayloadLtDnPoint():
    pass
def getBoxValue():
    pass

In [ ]:
# depth=3, row=4, column=2
data = np.zeros((3,4,2))
print data
for i in range(4):
    for j in range(2):
        r = random.choice([1,2,3,4])
        g = random.choice([1,2,3,4])
        b = random.choice([1,2,3,4])
        r, g, b = 255/r, 255/g, 255/b
        data[0][i][j] = r
        data[1][i][j] = g
        data[2][i][j] = b
print data

In [ ]:
data[1]=data[1][0,1]


In [29]:
data[2][0:1,0:1]

NameError: name 'data' is not defined

## study convolution

> Linear convolution of a and v :
$$(a*v)[n] = \sum_{m=-\infty}^{+\infty} a[m]v[n-m]$$

In [33]:
np.convolve([1,2,3],[2])

array([2, 4, 6])

In [20]:
np.convolve([1,2,3],[1,2])
# the convolution kernel is [2,1]

array([1, 4, 7, 6])

In [21]:
np.convolve([1,2,3], [0,1,0.5])

array([ 0. ,  1. ,  2.5,  4. ,  1.5])

In [22]:
np.convolve([1,2,3], [0,1,0.5], 'same')

array([ 1. ,  2.5,  4. ])

The two arrays are of the same length, so there is only one position where they completely overlap:

In [28]:
np.convolve([1,2,3], [0,1,0.5], 'valid')

array([ 2.5])